In [ ]:
#dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_meta_Home_and_Kitchen", trust_remote_code=True)

**Load Dataset. Shuffle & Sample**

In [22]:
import duckdb

meta_parquet = r"C:\Users\jthom\OneDrive\Documents\GitHub\MSBA-Capstone\meta_Home_and_Kitchen.parquet"
sample_out   = r"C:\Users\jthom\OneDrive\Documents\GitHub\MSBA-Capstone\meta_Home_and_Kitchen_sample_1pct_seed42.parquet"

PCT  = 1
SEED = 42

duckdb.sql("SET threads = 1;")  # ensures determinism for bernoulli/reservoir seeds

duckdb.sql(f"""
    COPY (
        SELECT *
        FROM read_parquet('{meta_parquet}')
        USING SAMPLE {PCT} PERCENT (bernoulli, {SEED})
    )
    TO '{sample_out}' (FORMAT 'parquet');
""")

print("Wrote seeded 1% sample to:", sample_out)


Wrote seeded 1% sample to: C:\Users\jthom\OneDrive\Documents\GitHub\MSBA-Capstone\meta_Home_and_Kitchen_sample_1pct_seed42.parquet


In [52]:
import duckdb

reviews_parquet = r"C:\Users\jthom\OneDrive\Documents\GitHub\MSBA-Capstone\Home_and_Kitchen.parquet"
sample_out      = r"C:\Users\jthom\OneDrive\Documents\GitHub\MSBA-Capstone\Home_and_Kitchen_verified_sample_30pct_seed42.parquet"

PCT  = 15
SEED = 42

duckdb.sql("SET threads = 1;")  # deterministic sampling with seed

duckdb.sql(f"""
    COPY (
        SELECT *
        FROM read_parquet('{reviews_parquet}')
        WHERE verified_purchase = TRUE
        USING SAMPLE {PCT} PERCENT (bernoulli, {SEED})
    )
    TO '{sample_out}' (FORMAT 'parquet');
""")

print("Wrote seeded verified 15% reviews sample to:", sample_out)


Wrote seeded verified 15% reviews sample to: C:\Users\jthom\OneDrive\Documents\GitHub\MSBA-Capstone\Home_and_Kitchen_verified_sample_30pct_seed42.parquet


In [1]:
import pandas as pd

reviews_sample = r"C:\Users\jthom\OneDrive\Documents\GitHub\MSBA-Capstone\Home_and_Kitchen_verified_sample_30pct_seed42.parquet"
meta_sample    = r"C:\Users\jthom\OneDrive\Documents\GitHub\MSBA-Capstone\meta_Home_and_Kitchen.parquet"

reviews_df = pd.read_parquet(reviews_sample)
meta_df    = pd.read_parquet(meta_sample)

print("Reviews sample shape:", reviews_df.shape)
print("Meta sample shape:   ", meta_df.shape)


Reviews sample shape: (9437426, 10)
Meta sample shape:    (3735584, 14)


In [2]:
# 1) (optional but recommended) drop duplicate products in meta
# keep the first occurrence of each parent_asin
meta_unique = meta_df.drop_duplicates(subset=["parent_asin"]).copy()

# 2) merge reviews with meta on parent_asin
final_df = reviews_df.merge(
    meta_unique,
    on="parent_asin",
    how="left",
    suffixes=("_review", "_meta")
)

print("Final shape:", final_df.shape)
final_df.head()


Final shape: (9437426, 23)


,rating,title_review,text,images_review,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase,...,rating_number,features,description,price,images_meta,videos,store,categories,details,bought_together
0,5.0,Excellent!,Lost my old one during my last move. This work...,[],B01CX1RIMQ,B08K8N5FB2,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,1581799806318,0,True,...,22106,[Better Than Your Grandma's Dough Blender - Yo...,[],7.99,[{'thumb': 'https://m.media-amazon.com/images/...,"[{'title': 'Watch Before You Buy REVIEW', 'url...",Spring Chef,"[Home & Kitchen, Kitchen & Dining, Bakeware, B...","[(Brand, ""Spring Chef""), (Color, ""Mint""), (Spe...",None
1,5.0,Soft and sturdy at the same time,I purchased this after having purchased some o...,[],B01GEDH4KU,B01GEDH4KU,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,1547919332140,0,True,...,5,[],[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],GOLD CASE,[],"[(Color, ""Gold""), (Brand, ""GOLD CASE""), (Numbe...",None
2,5.0,Totally. Obsessed. With dehydrating food. Than...,This is really a great dehydrator - and so eas...,[],B000FFVJ3C,B0B69XTPVL,AGGZ357AO26RQZVRLGU4D4N52DZQ,1354676658000,1,True,...,1106,[5 tray food dehydrator dries in hours instead...,[The NESCO FD-75PR Snackmaster Pro Food Dehydr...,NaN,[{'thumb': 'https://m.media-amazon.com/images/...,"[{'title': 'Heavyduty, great size but loud! Ne...",Nesco,"[Home & Kitchen, Kitchen & Dining, Small Appli...","[(Brand, ""Nesco""), (Material, ""Aluminum""), (Co...",None
3,4.0,Future-bubble coffeemaker is mostly a winner!,Hubs insisted on this coffee pot over the more...,[],B002FWQU3W,B002FWQU3W,AGGZ357AO26RQZVRLGU4D4N52DZQ,1354390111000,3,True,...,72,"[Auto pause and serve, Drip-free pouring, Prog...",[From Hamilton Beach the Illusion Coffeemaker ...,NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],Hamilton Beach,"[Home & Kitchen, Kitchen & Dining]","[(Brand, ""Hamilton Beach""), (Color, ""Black""), ...",None
4,4.0,It works.,"I wanted a small, inexpensive entryway shoe ra...",[],B002IPG46Y,B0778KV29D,AGGZ357AO26RQZVRLGU4D4N52DZQ,1341922513000,0,True,...,12294,[STACKING SHELF: Hardware included to securely...,[Take your storage & organization to the next ...,41.98,[{'thumb': 'https://m.media-amazon.com/images/...,[{'title': 'I got 2 of them easy to put togeth...,ClosetMaid,"[Home & Kitchen, Storage & Organization, Cloth...","[(Room Type, ""Bedroom""), (Number of Shelves, ""...",None


In [3]:
out_path = r"C:\Users\jthom\OneDrive\Documents\GitHub\MSBA-Capstone\hk_reviews_with_meta_sample_30pct.parquet"
final_df.to_parquet(out_path, index=False)
print("Wrote merged parquet to:", out_path)


Wrote merged parquet to: C:\Users\jthom\OneDrive\Documents\GitHub\MSBA-Capstone\hk_reviews_with_meta_sample_30pct.parquet


In [ ]:
import pandas as pd

merged_path = r"C:\Users\jthom\OneDrive\Documents\GitHub\MSBA-Capstone\hk_reviews_with_meta_sample_30pct.parquet"
final_df = pd.read_parquet(merged_path)

print("Final shape:", final_df.shape)
print(final_df.columns.tolist())

Final shape: (674079, 23)
['rating', 'title_review', 'text', 'images_review', 'asin', 'parent_asin', 'user_id', 'timestamp', 'helpful_vote', 'verified_purchase', 'main_category', 'title_meta', 'average_rating', 'rating_number', 'features', 'description', 'price', 'images_meta', 'videos', 'store', 'categories', 'details', 'bought_together']


In [4]:
meta_cols_merged = [c for c in final_df.columns if c.endswith("_meta")]

print("Meta columns in merged df:", meta_cols_merged)
print("Num meta cols:", len(meta_cols_merged))

missing_meta = final_df[meta_cols_merged].isna().all(axis=1).mean()
print(f"% reviews missing ALL meta fields: {missing_meta:.2%}")


Meta columns in merged df: ['title_meta', 'images_meta']
Num meta cols: 2
% reviews missing ALL meta fields: 0.00%


In [5]:
matched_any_meta = 1 - missing_meta
print(f"% reviews with SOME meta: {matched_any_meta:.2%}")
print("Unique products in final data:", final_df["parent_asin"].nunique())

% reviews with SOME meta: 100.00%
Unique products in final data: 1510827


In [6]:
# % reviews with SOME meta (kept as-is)
matched_any_meta = 1 - missing_meta
print(f"% reviews with SOME meta: {matched_any_meta:.2%}")

# unique users in final data
print("Unique users in final data:", final_df["user_id"].nunique())


% reviews with SOME meta: 100.00%
Unique users in final data: 6726153


In [7]:
u_counts = reviews_df["user_id"].value_counts()
print(u_counts.quantile([.5, .75, .9, .95, .99]))


0.50    1.0
0.75    1.0
0.90    2.0
0.95    3.0
0.99    6.0
Name: count, dtype: float64


In [8]:
import pandas as pd

reviews_sample_path = reviews_sample  # if this is your string path
reviews_sample_df = pd.read_parquet(reviews_sample_path)

N = len(reviews_sample_df)
U = reviews_sample_df["user_id"].nunique()
I = reviews_sample_df["parent_asin"].nunique()

print("Interactions:", N)
print("Unique users:", U)
print("Unique items:", I)
print("Interactions per user:", N / U)
print("Interactions per item:", N / I)


Interactions: 9437426
Unique users: 6726153
Unique items: 1510827
Interactions per user: 1.4030941609564933
Interactions per item: 6.2465298806547676


In [9]:
reviews_df["user_id"] = reviews_df["user_id"].astype("category")
reviews_df["parent_asin"] = reviews_df["parent_asin"].astype("category")


In [10]:
def k_core_filter(df, user_col="user_id", item_col="parent_asin", k_user=3, k_item=3, verbose=True, max_iters=100):
    """
    Iteratively filter to a k-core on the user–item interaction graph.
    Keeps only users with >= k_user interactions and items with >= k_item interactions.
    Repeats until stable.
    """
    cur = df
    prev_shape = None
    it = 0

    while prev_shape != cur.shape and it < max_iters:
        it += 1
        prev_shape = cur.shape

        # keep users with enough interactions
        user_counts = cur[user_col].value_counts()
        good_users = user_counts[user_counts >= k_user].index
        cur = cur[cur[user_col].isin(good_users)]

        # keep items with enough interactions
        item_counts = cur[item_col].value_counts()
        good_items = item_counts[item_counts >= k_item].index
        cur = cur[cur[item_col].isin(good_items)]

        if verbose:
            print(
                f"iter {it}: rows={len(cur)}, "
                f"users={cur[user_col].nunique()}, "
                f"items={cur[item_col].nunique()}"
            )

    if it >= max_iters:
        print(f"Stopped at max_iters={max_iters} before fully stabilizing.")

    return cur

core3 = k_core_filter(reviews_df, k_user=3, k_item=3)

print(core3.shape,
      core3["user_id"].nunique(),
      core3["parent_asin"].nunique())


iter 1: rows=1630553, users=546755, items=153642
iter 2: rows=1186114, users=323173, items=117232
iter 3: rows=1119936, users=294663, items=111583
iter 4: rows=1107295, users=289329, items=110554
iter 5: rows=1104870, users=288310, items=110356
iter 6: rows=1104403, users=288117, items=110315
iter 7: rows=1104325, users=288083, items=110309
iter 8: rows=1104307, users=288074, items=110309
iter 9: rows=1104307, users=288074, items=110309
(1104307, 10) 288074 110309


In [11]:
N = len(core3)
U = core3["user_id"].nunique()
I = core3["parent_asin"].nunique()

print("Interactions per user:", N / U)
print("Interactions per item:", N / I)


Interactions per user: 3.833414331039941
Interactions per item: 10.011032644661814


In [12]:
out_path2 = r"C:\Users\jthom\OneDrive\Documents\GitHub\MSBA-Capstone\core_hk_reviews_with_meta_sample_30pct.parquet"
core3.to_parquet(out_path2, index=False)
print("Wrote merged parquet to:", out_path2)

Wrote merged parquet to: C:\Users\jthom\OneDrive\Documents\GitHub\MSBA-Capstone\core_hk_reviews_with_meta_sample_30pct.parquet


In [2]:
import pandas as pd

out_path2 = r"C:\Users\jthom\OneDrive\Documents\GitHub\MSBA-Capstone\core_hk_reviews_with_meta_sample_30pct.parquet"
df2 = pd.read_parquet(out_path2)
print("Row count:", len(df2))

Row count: 1104307


**Full-dataset distributions in Spark**

In [36]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count

spark = SparkSession.builder.appName("AmazonFinalEDA").getOrCreate()

# ✅ path to your merged final dataset
final_path = r"C:\Users\jthom\OneDrive\Documents\GitHub\MSBA-Capstone\hk_reviews_with_meta_sample.parquet"
df_final = spark.read.parquet(final_path)

# sanity check
df_final.printSchema()

# pick the right rating column
rating_candidates = ["rating", "overall", "rating_review"]
rating_col = next((c for c in rating_candidates if c in df_final.columns), None)

if rating_col is None:
    raise ValueError(f"No rating column found. Columns are: {df_final.columns}")

df_final.select(rating_col).show(5)

# rating distribution
pop_rating_spark = (
    df_final.groupBy(rating_col)
            .agg(count("*").alias("count"))
            .orderBy(col(rating_col))
)

pop_rating = pop_rating_spark.toPandas()
pop_rating["prop"] = pop_rating["count"] / pop_rating["count"].sum()

pop_rating


KeyboardInterrupt: 

In [ ]:
from pyspark.sql.functions import coalesce, lit

pop_vp = (
    df_full.select(coalesce(col("verified_purchase"), lit(False)).alias("verified_purchase"))
           .groupBy("verified_purchase")
           .agg(count("*").alias("count"))
           .toPandas()
)
pop_vp["prop"] = pop_vp["count"] / pop_vp["count"].sum()
pop_vp


**Sample distributions in pandas**

In [ ]:
import pandas as pd

# Sample (or subset) of reviews loaded into pandas for detailed EDA
pdf = pd.read_parquet("verified_reviews_100k.parquet")

samp_rating = pdf["rating"].value_counts().sort_index().reset_index()
samp_rating.columns = ["rating", "count"]
samp_rating["prop"] = samp_rating["count"] / samp_rating["count"].sum()

samp_vp = pdf["verified_purchase"].fillna(False).value_counts().reset_index()
samp_vp.columns = ["verified_purchase", "count"]
samp_vp["prop"] = samp_vp["count"] / samp_vp["count"].sum()


**Chi-square test for rating distribution**

In [ ]:
from scipy.stats import chisquare
import numpy as np

# align categories 1..5
cats = pop_rating["rating"].tolist()

obs = samp_rating.set_index("rating").reindex(cats)["count"].fillna(0).values
exp_props = pop_rating.set_index("rating").reindex(cats)["prop"].values
exp = exp_props * obs.sum()

chi2, p = chisquare(f_obs=obs, f_exp=exp)
chi2, p


- p > 0.05 → no evidence sample differs from population (good sign).
- p ≤ 0.05 → sample likely biased on ratings; increase buffer/N or re-sample.

**Two-proportion z-test for verified purchase rate**

In [ ]:
from statsmodels.stats.proportion import proportions_ztest

p_pop = pop_vp.set_index("verified_purchase").loc[True, "prop"] if True in pop_vp["verified_purchase"].values else 0.0
p_samp = samp_vp.set_index("verified_purchase").loc[True, "prop"] if True in samp_vp["verified_purchase"].values else 0.0

count_samp = samp_vp.set_index("verified_purchase").loc[True, "count"] if True in samp_vp["verified_purchase"].values else 0
n_samp = samp_vp["count"].sum()

# expected count under population prop
count_pop_equiv = p_pop * n_samp

stat, p = proportions_ztest([count_samp, count_pop_equiv], [n_samp, n_samp])
stat, p


In [ ]:
import matplotlib.pyplot as plt

# rating distribution
pdf["rating"].value_counts().sort_index().plot(kind="bar")
plt.title("Rating Distribution (sample)")
plt.xlabel("Stars"); plt.ylabel("Count")
plt.show()

# verified purchase share (if column exists)
if "verified_purchase" in pdf.columns:
    pdf["verified_purchase"].value_counts().plot(kind="bar")
    plt.title("Verified Purchase Share (sample)")
    plt.show()

# review length
if "text" in pdf.columns:
    pdf["text_len"] = pdf["text"].astype(str).str.len()
    plt.hist(pdf["text_len"], bins=60, log=True)
    plt.title("Review Text Length (sample)")
    plt.xlabel("Chars"); plt.ylabel("Count (log)")
    plt.show()


**User-level EDA with merged reviews and product metadata**

In [ ]:
import pandas as pd

meta_path = "meta_Home_and_Kitchen.parquet"  # product metadata
meta_df = pd.read_parquet(meta_path)
meta_df.head()

In [ ]:
reviews_path = "verified_reviews_100k.parquet"  # sample of user reviews/purchases
reviews_df = pd.read_parquet(reviews_path)
reviews_df.head()

In [ ]:
# merge reviews (user interactions) with product metadata on parent_asin
product_cols = [
    "parent_asin",
    "title",
    "main_category",
    "average_rating",
    "rating_number",
    "price",
]
product_cols = [c for c in product_cols if c in meta_df.columns]

merged_df = reviews_df.merge(
    meta_df[product_cols],
    on="parent_asin",
    how="left",
)

merged_df.head()

In [ ]:
# basic user-level purchase behavior
user_counts = (
    merged_df
    .groupby("user_id")["parent_asin"]
    .count()
    .sort_values(ascending=False)
)
user_counts.head(10)

In [ ]:
# user-level rating behavior (if rating column is present)
if "rating" in merged_df.columns:
    user_stats = (
        merged_df
        .groupby("user_id")["rating"]
        .agg(["count", "mean"])
        .sort_values("count", ascending=False)
    )
    display(user_stats.head(10))
else:
    print("Column 'rating' not found in merged_df; adjust to your rating column name.")